In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import sys
from pathlib import Path
import anndata as ad
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import seaborn as sns
from matplotlib import pyplot as plt

sys.path.insert(0, "/home/ylu/project/spateo-release_new")
import spateo_dev as st
sys.path.insert(0, "/home/ylu/project")
from utils import *

os.system('/usr/bin/Xvfb :99 -screen 0 1024x768x24 &')
os.environ['DISPLAY'] = ':99'

%load_ext autoreload
%autoreload 2

2024-08-14 21:00:14.344617: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-14 21:00:14.509607: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-08-14 21:00:14.509634: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-08-14 21:00:15.233301: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directo

In [2]:
data_folder = "../../data/BARseq/BARseq_Perturbed/"
# Path(results_folder).mkdir(parents=True, exist_ok=True)
method = 'Moscot'

In [3]:
slices = [ad.read_h5ad(data_folder + "slice_{:0>2d}.h5ad".format(i)) for i in range(1, 41)]

In [4]:
transformation = np.load(f'./results/processed_results/{method}_transformation.npy', allow_pickle=True).item()
align_Rotation = transformation['align_Rotation']
align_translation = transformation['align_translation']

In [5]:
slices[0]

AnnData object with n_obs × n_vars = 34029 × 114
    obs: 'cluster', 'slice_id', 'simplify_cluster'
    obsm: 'align_spatial', 'perturbed_spatial', 'spatial', 'spatial_CCF', 'spatial_raw'

In [6]:
from tqdm import tqdm
cur_R = np.eye(2)
cur_t = np.zeros(2)
for i in tqdm(range(len(slices))):
    slices[i].obsm['align_spatial'] = slices[i].obsm['perturbed_spatial'] @ cur_R.T + cur_t
    if i < len(slices)-1:
        R = align_Rotation[i]
        t = align_translation[i]
        cur_t = t @ cur_R.T + cur_t
        cur_R = cur_R @ R

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 783.50it/s]


In [7]:
z_values = []
for i in range(len(slices)):
    z_values.append(np.mean(slices[i].obsm['spatial_CCF'][:,2]))

In [8]:
import pyvista as pv
brain_mesh = pv.read('../../data/HalfAdultMouseBrain.vtk')

In [9]:
brain_mesh_align = st.tdr.scale_model(model=brain_mesh, scale_factor=40, scale_center=None, inplace=False)
brain_mesh_align = st.tdr.rotate_model(model=brain_mesh_align, angle=(0, 0, 90), rotate_center= brain_mesh.center, inplace=False)

In [10]:
z_shift_correction = st.align.z_shift_correction(
    models=slices,
    mesh=brain_mesh_align,
    spatial_key='align_spatial',
    key_added='align_spatial_z_corrected',
    z_values=z_values,
)

In [11]:
z_shift_correction.extract_contours(method='alpha', smoothing=True, window_size=3, outlier_threshold=30, alpha=0.2, n_sampling=10000)

|-----> [Extract contours of slices, method: alpha.] in progress: 100.0000%
|-----> [Extract contours of slices, method: alpha.] finished [80.4079s]


In [ ]:
init_parameters = np.array([0,0,0,0,1])
best_parameters, _ = z_shift_correction.start_discrete_optimization(
    init_parameters=init_parameters,
    max_rotation_angle=5,
    max_translation_scale=0.1,
    max_scaling=1.05,
    labelNum=8,
    nIters=10,
    multi_process=True,
)

Current iter: 0, parameters: [0 0 0 0 1] with loss: 100000000.0
evaluate 10 pairs
Current pair: 0/10


/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning:

Mean of empty slice.

/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/numpy/core/_methods.py:184: RuntimeWarning:

invalid value encountered in divide

/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/numpy/core/_methods.py:192: RuntimeWarning:

invalid value encountered in scalar divide

/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning:

Mean of empty slice.

/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/numpy/core/_methods.py:184: RuntimeWarning:

invalid value encountered in divide

/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/numpy/core/_methods.py:192: RuntimeWarning:

invalid value encountered in scalar divide

/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning:

Mean of empty slice.

/home/ylu/anaconda3/envs/

Current pair: 1/10


/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning:

Mean of empty slice.

/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/numpy/core/_methods.py:184: RuntimeWarning:

invalid value encountered in divide

/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/numpy/core/_methods.py:192: RuntimeWarning:

invalid value encountered in scalar divide

/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning:

Mean of empty slice.

/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/numpy/core/_methods.py:184: RuntimeWarning:

invalid value encountered in divide

/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/numpy/core/_methods.py:192: RuntimeWarning:

invalid value encountered in scalar divide

/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning:

Mean of empty slice.

/home/ylu/anaconda3/envs/

Current pair: 2/10


...

Updating parameters!
Current iter: 3, parameters: [ -0.71428571  10.71428571  -7.85714286 -49.89134494   0.96575015] with loss: 0.19743330803425646
evaluate 10 pairs
Current pair: 0/10
Current pair: 1/10
Current pair: 2/10
Current pair: 3/10


/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning:

Mean of empty slice.

/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/numpy/core/_methods.py:184: RuntimeWarning:

invalid value encountered in divide

/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/numpy/core/_methods.py:192: RuntimeWarning:

invalid value encountered in scalar divide



Current pair: 4/10
Current pair: 5/10
Current pair: 6/10
Current pair: 7/10
Current pair: 8/10
Current pair: 9/10


/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning:

Mean of empty slice.

/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/numpy/core/_methods.py:184: RuntimeWarning:

invalid value encountered in divide

/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/numpy/core/_methods.py:192: RuntimeWarning:

invalid value encountered in scalar divide

/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning:

Mean of empty slice.

/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/numpy/core/_methods.py:184: RuntimeWarning:

invalid value encountered in divide

/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/numpy/core/_methods.py:192: RuntimeWarning:

invalid value encountered in scalar divide

/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning:

Mean of empty slice.

/home/ylu/anaconda3/envs/

Updating parameters!
Current iter: 4, parameters: [ -5.          10.71428571  -6.42857143 -52.48026041   0.96575015] with loss: 0.16191800154147393
evaluate 10 pairs
Current pair: 0/10
Current pair: 1/10
Current pair: 2/10
Current pair: 3/10
Current pair: 4/10
Current pair: 5/10
Current pair: 6/10
Current pair: 7/10
Current pair: 8/10
Current pair: 9/10


In [ ]:
z_shift_correction.get_z_shift(smooth=False, allow_rotation=True)

In [ ]:
z_shift_correction.z_shift_correction(
    models=slices,
)

In [ ]:
align_Rotation = []
align_translation = []
for s in slices:
    R, t = solve_RT_by_correspondence(s.obsm['align_spatial_z_corrected'], s.obsm['perturbed_spatial'])
    align_Rotation.append(R)
    align_translation.append(t)
np.save(f'./mesh_corr_results/{method}_mesh_transformation.npy', {'align_Rotation':align_Rotation, 'align_translation':align_translation, 'best_parameters': z_shift_correction.best_parameters}, allow_pickle=True)